In [1]:
import os
from dotenv import load_dotenv 
from openai import OpenAI
from IPython.display import Markdown, display
import pymupdf

In [2]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [4]:
def extract_pdf_content(pdf_path):
    pdf = pymupdf.open(pdf_path)
    print(pdf.get_page_numbers)
    print(pdf.metadata)

    text = ''
    for page in pdf:
        text += page.get_text() + "\n"
    print(f"Full Text: {text}")

    pdf.close()

    return text

In [5]:
system_prompt = """ You are an intelligent assistant that can summarize text.
                    You will be given a text and you need to summarize it in a few sentences kepping all the important information.
                    Do not provide any additional information which is not there in the provided text.
                    Highligh the important points for a quick read through."""

user_prompt = "Here are the content of the pdf. Provide a summary of the content."

user_prompt += f"\n\n{extract_pdf_content('EC2_WordPress_Migration_machmacurry.pdf')}"


<bound method Document.get_page_numbers of Document('EC2_WordPress_Migration_machmacurry.pdf')>
{'format': 'PDF 1.4', 'title': '(anonymous)', 'author': '(anonymous)', 'subject': '(unspecified)', 'keywords': '', 'creator': '(unspecified)', 'producer': 'ReportLab PDF Library - www.reportlab.com', 'creationDate': "D:20260102210918+00'00'", 'modDate': "D:20260102210918+00'00'", 'trapped': '', 'encryption': None}
Full Text: EC2 WordPress Migration Guide
machmacurry.com
This document describes, in detail, the complete end-to-end process followed to migrate the
WordPress website machmacurry.com from AWS Account A (EC2-A) to AWS Account B
(EC2-B). It includes the reasoning behind each step, common issues encountered, and the clean
best-practice approach used to bring the site live successfully.
1. Migration Objective
The primary goal was to decommission AWS Account A completely while preserving the WordPress
website, database, media, themes, plugins, and domain configuration. The website had t

In [7]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}    
]

client = OpenAI(api_key=api_key)
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    frequency_penalty=0
)

summary = response.choices[0].message.content

In [8]:
display(Markdown(summary))

**EC2 WordPress Migration Guide Summary**

This guide details the process of migrating the WordPress website machmacurry.com from AWS Account A (EC2-A) to AWS Account B (EC2-B). 

**Key Points:**
1. **Migration Objective**: Decommission EC2-A, retaining full website functionality in EC2-B with HTTPS and correct DNS.
2. **Source Environment**: 
   - EC2-A had an Ubuntu instance with Nginx and Bitnami WordPress, managed domain via Route 53, and SSL via Let’s Encrypt.
3. **Target Environment**: 
   - A fresh EC2-B instance with Ubuntu 22.04 and manually deployed WordPress.
4. **Backup Process**: 
   - Full database and WordPress files backup using mysqldump and SCP/PSCP for local transfer.
5. **EC2-B Setup**: 
   - New instance launched, installing all necessary components and configuring security groups.
6. **WordPress Restoration**: 
   - Files restored and database imported; configurations updated in wp-config.php.
7. **Issues Resolved**: 
   - Fixed login issues by correcting Nginx and database values.
8. **DNS Migration**: 
   - Updated nameservers or A records to point to the new EC2 Elastic IP.
9. **SSL Setup**: 
   - Installed Certbot for HTTPS and ensured automatic renewal.
10. **Final Validation**: 
   - Verified website functionality including admin access and SSL.
11. **Decommissioning Account A**: 
   - Verified operational stability in EC2-B before safely terminating EC2-A and closing the account.

The migration successfully establishes machmacurry.com in AWS Account B with an optimized setup.